# NTU SC1015 - Yelp Dataset Preparation
[Yelp Dataset](https://www.yelp.com/dataset) is a subset of Yelp's internal data for academic research:

![Yelp Dataset ERD](data:image/svg+xml,%3Csvg%20aria-roledescription%3D%22er%22%20role%3D%22graphics-document%20document%22%20viewBox%3D%220%20-62%20735.05078125%20926.5%22%20style%3D%22max-width%3A%20100%25%3B%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22100%25%22%20id%3D%22graph-div%22%20height%3D%22100%25%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%3E%3Cstyle%3E%40import%20url%28%22https%3A%2F%2Fcdnjs.cloudflare.com%2Fajax%2Flibs%2Ffont-awesome%2F6.5.1%2Fcss%2Fall.min.css%22%29%3B%27%3C%2Fstyle%3E%3Cstyle%3E%23graph-div%7Bfont-family%3A%22trebuchet%20ms%22%2Cverdana%2Carial%2Csans-serif%3Bfont-size%3A16px%3Bfill%3A%23333%3B%7D%23graph-div%20.error-icon%7Bfill%3A%23552222%3B%7D%23graph-div%20.error-text%7Bfill%3A%23552222%3Bstroke%3A%23552222%3B%7D%23graph-div%20.edge-thickness-normal%7Bstroke-width%3A2px%3B%7D%23graph-div%20.edge-thickness-thick%7Bstroke-width%3A3.5px%3B%7D%23graph-div%20.edge-pattern-solid%7Bstroke-dasharray%3A0%3B%7D%23graph-div%20.edge-pattern-dashed%7Bstroke-dasharray%3A3%3B%7D%23graph-div%20.edge-pattern-dotted%7Bstroke-dasharray%3A2%3B%7D%23graph-div%20.marker%7Bfill%3A%23333333%3Bstroke%3A%23333333%3B%7D%23graph-div%20.marker.cross%7Bstroke%3A%23333333%3B%7D%23graph-div%20svg%7Bfont-family%3A%22trebuchet%20ms%22%2Cverdana%2Carial%2Csans-serif%3Bfont-size%3A16px%3B%7D%23graph-div%20.entityBox%7Bfill%3A%23ECECFF%3Bstroke%3A%239370DB%3B%7D%23graph-div%20.attributeBoxOdd%7Bfill%3A%23ffffff%3Bstroke%3A%239370DB%3B%7D%23graph-div%20.attributeBoxEven%7Bfill%3A%23f2f2f2%3Bstroke%3A%239370DB%3B%7D%23graph-div%20.relationshipLabelBox%7Bfill%3Ahsl%2880%2C%20100%25%2C%2096.2745098039%25%29%3Bopacity%3A0.7%3Bbackground-color%3Ahsl%2880%2C%20100%25%2C%2096.2745098039%25%29%3B%7D%23graph-div%20.relationshipLabelBox%20rect%7Bopacity%3A0.5%3B%7D%23graph-div%20.relationshipLine%7Bstroke%3A%23333333%3B%7D%23graph-div%20.entityTitleText%7Btext-anchor%3Amiddle%3Bfont-size%3A18px%3Bfill%3A%23333%3B%7D%23graph-div%20%23MD_PARENT_START%7Bfill%3A%23f5f5f5%21important%3Bstroke%3A%23333333%21important%3Bstroke-width%3A1%3B%7D%23graph-div%20%23MD_PARENT_END%7Bfill%3A%23f5f5f5%21important%3Bstroke%3A%23333333%21important%3Bstroke-width%3A1%3B%7D%23graph-div%20%3Aroot%7B--mermaid-font-family%3A%22trebuchet%20ms%22%2Cverdana%2Carial%2Csans-serif%3B%7D%3C%2Fstyle%3E%3Cg%3E%3C%2Fg%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%22240%22%20markerWidth%3D%22190%22%20refY%3D%227%22%20refX%3D%220%22%20id%3D%22MD_PARENT_START%22%3E%3Cpath%20d%3D%22M%2018%2C7%20L9%2C13%20L1%2C7%20L9%2C1%20Z%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2228%22%20markerWidth%3D%2220%22%20refY%3D%227%22%20refX%3D%2219%22%20id%3D%22MD_PARENT_END%22%3E%3Cpath%20d%3D%22M%2018%2C7%20L9%2C13%20L1%2C7%20L9%2C1%20Z%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2218%22%20markerWidth%3D%2218%22%20refY%3D%229%22%20refX%3D%220%22%20id%3D%22ONLY_ONE_START%22%3E%3Cpath%20d%3D%22M9%2C0%20L9%2C18%20M15%2C0%20L15%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2218%22%20markerWidth%3D%2218%22%20refY%3D%229%22%20refX%3D%2218%22%20id%3D%22ONLY_ONE_END%22%3E%3Cpath%20d%3D%22M3%2C0%20L3%2C18%20M9%2C0%20L9%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2218%22%20markerWidth%3D%2230%22%20refY%3D%229%22%20refX%3D%220%22%20id%3D%22ZERO_OR_ONE_START%22%3E%3Ccircle%20r%3D%226%22%20cy%3D%229%22%20cx%3D%2221%22%20fill%3D%22white%22%20stroke%3D%22gray%22%3E%3C%2Fcircle%3E%3Cpath%20d%3D%22M9%2C0%20L9%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2218%22%20markerWidth%3D%2230%22%20refY%3D%229%22%20refX%3D%2230%22%20id%3D%22ZERO_OR_ONE_END%22%3E%3Ccircle%20r%3D%226%22%20cy%3D%229%22%20cx%3D%229%22%20fill%3D%22white%22%20stroke%3D%22gray%22%3E%3C%2Fcircle%3E%3Cpath%20d%3D%22M21%2C0%20L21%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2236%22%20markerWidth%3D%2245%22%20refY%3D%2218%22%20refX%3D%2218%22%20id%3D%22ONE_OR_MORE_START%22%3E%3Cpath%20d%3D%22M0%2C18%20Q%2018%2C0%2036%2C18%20Q%2018%2C36%200%2C18%20M42%2C9%20L42%2C27%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2236%22%20markerWidth%3D%2245%22%20refY%3D%2218%22%20refX%3D%2227%22%20id%3D%22ONE_OR_MORE_END%22%3E%3Cpath%20d%3D%22M3%2C9%20L3%2C27%20M9%2C18%20Q27%2C0%2045%2C18%20Q27%2C36%209%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2236%22%20markerWidth%3D%2257%22%20refY%3D%2218%22%20refX%3D%2218%22%20id%3D%22ZERO_OR_MORE_START%22%3E%3Ccircle%20r%3D%226%22%20cy%3D%2218%22%20cx%3D%2248%22%20fill%3D%22white%22%20stroke%3D%22gray%22%3E%3C%2Fcircle%3E%3Cpath%20d%3D%22M0%2C18%20Q18%2C0%2036%2C18%20Q18%2C36%200%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cdefs%3E%3Cmarker%20orient%3D%22auto%22%20markerHeight%3D%2236%22%20markerWidth%3D%2257%22%20refY%3D%2218%22%20refX%3D%2239%22%20id%3D%22ZERO_OR_MORE_END%22%3E%3Ccircle%20r%3D%226%22%20cy%3D%2218%22%20cx%3D%229%22%20fill%3D%22white%22%20stroke%3D%22gray%22%3E%3C%2Fcircle%3E%3Cpath%20d%3D%22M21%2C18%20Q39%2C0%2057%2C18%20Q39%2C36%2021%2C18%22%20fill%3D%22none%22%20stroke%3D%22gray%22%3E%3C%2Fpath%3E%3C%2Fmarker%3E%3C%2Fdefs%3E%3Cpath%20style%3D%22stroke%3A%20gray%3B%20fill%3A%20none%3B%22%20marker-start%3D%22url%28%23ZERO_OR_MORE_START%29%22%20marker-end%3D%22url%28%23ONLY_ONE_END%29%22%20d%3D%22M523.191%2C191.554L505.135%2C209.154C487.079%2C226.755%2C450.967%2C261.955%2C413.359%2C311.03C375.75%2C360.105%2C336.645%2C423.054%2C317.092%2C454.528L297.539%2C486.003%22%20class%3D%22er%20relationshipLine%22%3E%3C%2Fpath%3E%3Cpath%20style%3D%22stroke%3A%20gray%3B%20fill%3A%20none%3B%22%20marker-start%3D%22url%28%23ZERO_OR_MORE_START%29%22%20marker-end%3D%22url%28%23ONLY_ONE_END%29%22%20d%3D%22M642.145%2C230.869L648.898%2C241.917C655.652%2C252.965%2C669.16%2C275.06%2C674.519%2C294.442C679.878%2C313.823%2C677.088%2C330.49%2C675.693%2C338.823L674.298%2C347.156%22%20class%3D%22er%20relationshipLine%22%3E%3C%2Fpath%3E%3Cpath%20style%3D%22stroke%3A%20gray%3B%20fill%3A%20none%3B%22%20marker-start%3D%22url%28%23ZERO_OR_MORE_START%29%22%20marker-end%3D%22url%28%23ONLY_ONE_END%29%22%20d%3D%22M305.138%2C202.125L295.456%2C217.964C285.773%2C233.802%2C266.408%2C265.479%2C255.563%2C300.909C244.717%2C336.339%2C242.391%2C375.521%2C241.228%2C395.112L240.065%2C414.703%22%20class%3D%22er%20relationshipLine%22%3E%3C%2Fpath%3E%3Cpath%20style%3D%22stroke%3A%20gray%3B%20fill%3A%20none%3B%22%20marker-start%3D%22url%28%23ZERO_OR_MORE_START%29%22%20marker-end%3D%22url%28%23ONLY_ONE_END%29%22%20d%3D%22M417.364%2C202.125L433.613%2C217.964C449.861%2C233.802%2C482.358%2C265.479%2C504.512%2C296.288C526.665%2C327.096%2C538.475%2C357.036%2C544.38%2C372.006L550.285%2C386.976%22%20class%3D%22er%20relationshipLine%22%3E%3C%2Fpath%3E%3Cpath%20style%3D%22stroke%3A%20gray%3B%20fill%3A%20none%3B%22%20marker-start%3D%22url%28%23ZERO_OR_MORE_START%29%22%20marker-end%3D%22url%28%23ONLY_ONE_END%29%22%20d%3D%22M79.313%2C168.352L79.313%2C189.819C79.313%2C211.286%2C79.313%2C254.221%2C92.941%2C302.826C106.57%2C351.431%2C133.828%2C405.705%2C147.457%2C432.842L161.086%2C459.979%22%20class%3D%22er%20relationshipLine%22%3E%3C%2Fpath%3E%3Cg%20transform%3D%22translate%28161.0859375%2C414.703125%20%29%22%20id%3D%22entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874%22%3E%3Crect%20height%3D%22362.25%22%20width%3D%22136.453125%22%20y%3D%220%22%20x%3D%220%22%20class%3D%22er%20entityBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20text-anchor%3A%20middle%3B%20font-size%3A%2012px%3B%22%20transform%3D%22translate%2868.2265625%2C12.2578125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874%22%20class%3D%22er%20entityLabel%22%3EBusiness%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2224.515625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-1-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%2224.515625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-1-name%22%20class%3D%22er%20entityLabel%22%3Ebusiness_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2224.515625%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-1-key%22%20class%3D%22er%20entityLabel%22%3EPK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2247.03125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-2-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%2247.03125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-2-name%22%20class%3D%22er%20entityLabel%22%3Ename%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2247.03125%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-2-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2269.546875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-3-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%2269.546875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-3-name%22%20class%3D%22er%20entityLabel%22%3Eaddress%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2269.546875%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-3-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2292.0625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-4-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%2292.0625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-4-name%22%20class%3D%22er%20entityLabel%22%3Ecity%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2292.0625%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-4-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22114.578125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-5-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22114.578125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-5-name%22%20class%3D%22er%20entityLabel%22%3Estate%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22114.578125%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-5-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22137.09375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-6-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22137.09375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-6-name%22%20class%3D%22er%20entityLabel%22%3Estate%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22137.09375%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-6-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22159.609375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-7-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22159.609375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-7-name%22%20class%3D%22er%20entityLabel%22%3Epotal_code%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22159.609375%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-7-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22182.125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-8-type%22%20class%3D%22er%20entityLabel%22%3Efloat%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22182.125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-8-name%22%20class%3D%22er%20entityLabel%22%3Elatitude%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22182.125%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-8-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22204.640625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-9-type%22%20class%3D%22er%20entityLabel%22%3Efloat%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22204.640625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-9-name%22%20class%3D%22er%20entityLabel%22%3Elongitude%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22204.640625%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-9-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22227.15625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-10-type%22%20class%3D%22er%20entityLabel%22%3Efloat%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22227.15625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-10-name%22%20class%3D%22er%20entityLabel%22%3Estars%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22227.15625%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-10-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22249.671875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-11-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22249.671875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-11-name%22%20class%3D%22er%20entityLabel%22%3Ereview_count%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22249.671875%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-11-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22272.1875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-12-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22272.1875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-12-name%22%20class%3D%22er%20entityLabel%22%3Eis_open%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22272.1875%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-12-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22294.703125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-13-type%22%20class%3D%22er%20entityLabel%22%3Edict%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22294.703125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-13-name%22%20class%3D%22er%20entityLabel%22%3Eattributes%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22294.703125%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-13-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22317.21875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-14-type%22%20class%3D%22er%20entityLabel%22%3Estring%5B%5D%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22317.21875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-14-name%22%20class%3D%22er%20entityLabel%22%3Ecategories%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22317.21875%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-14-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22339.734375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-15-type%22%20class%3D%22er%20entityLabel%22%3Edict%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2271.65625%22%20y%3D%22339.734375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-15-name%22%20class%3D%22er%20entityLabel%22%3Ehours%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22339.734375%22%20x%3D%22114.890625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28119.890625%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Business-6d5256e9-9330-5277-b783-ed06f8c0f874-attr-15-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20transform%3D%22translate%28523.19140625%2C20%20%29%22%20id%3D%22entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd%22%3E%3Crect%20height%3D%22227.15625%22%20width%3D%22118.953125%22%20y%3D%220%22%20x%3D%220%22%20class%3D%22er%20entityBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20text-anchor%3A%20middle%3B%20font-size%3A%2012px%3B%22%20transform%3D%22translate%2859.4765625%2C12.2578125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd%22%20class%3D%22er%20entityLabel%22%3EReview%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2224.515625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-1-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2224.515625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-1-name%22%20class%3D%22er%20entityLabel%22%3Ereview_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2224.515625%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-1-key%22%20class%3D%22er%20entityLabel%22%3EPK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2247.03125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-2-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2247.03125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-2-name%22%20class%3D%22er%20entityLabel%22%3Euser_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2247.03125%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-2-key%22%20class%3D%22er%20entityLabel%22%3EFK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2269.546875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-3-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2269.546875%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-3-name%22%20class%3D%22er%20entityLabel%22%3Ebusiness_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2269.546875%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-3-key%22%20class%3D%22er%20entityLabel%22%3EFK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2292.0625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-4-type%22%20class%3D%22er%20entityLabel%22%3Efloat%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2292.0625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-4-name%22%20class%3D%22er%20entityLabel%22%3Estars%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2292.0625%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-4-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22114.578125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-5-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%22114.578125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-5-name%22%20class%3D%22er%20entityLabel%22%3Edate%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22114.578125%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-5-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22137.09375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-6-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%22137.09375%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-6-name%22%20class%3D%22er%20entityLabel%22%3Etext%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22137.09375%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-6-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22159.609375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-7-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%22159.609375%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-7-name%22%20class%3D%22er%20entityLabel%22%3Euseful%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22159.609375%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-7-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22182.125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-8-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%22182.125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-8-name%22%20class%3D%22er%20entityLabel%22%3Efunny%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22182.125%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-8-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22204.640625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-9-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%22204.640625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-9-name%22%20class%3D%22er%20entityLabel%22%3Ecool%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22204.640625%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Review-77c92a79-8cff-50c0-9604-f9135763cccd-attr-9-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20transform%3D%22translate%28550.28515625%2C347.15625%20%29%22%20id%3D%22entity-User-818c511a-4a32-5484-ba70-875d765a9175%22%3E%3Crect%20height%3D%22497.34375%22%20width%3D%22164.765625%22%20y%3D%220%22%20x%3D%220%22%20class%3D%22er%20entityBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20text-anchor%3A%20middle%3B%20font-size%3A%2012px%3B%22%20transform%3D%22translate%2882.3828125%2C12.2578125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175%22%20class%3D%22er%20entityLabel%22%3EUser%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2224.515625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-1-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%2224.515625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-1-name%22%20class%3D%22er%20entityLabel%22%3Euser_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2224.515625%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-1-key%22%20class%3D%22er%20entityLabel%22%3EPK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2247.03125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-2-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%2247.03125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-2-name%22%20class%3D%22er%20entityLabel%22%3Ename%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2247.03125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-2-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2269.546875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-3-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%2269.546875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-3-name%22%20class%3D%22er%20entityLabel%22%3Ereview_count%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2269.546875%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-3-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%2292.0625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-4-type%22%20class%3D%22er%20entityLabel%22%3Estring%5B%5D%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%2292.0625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-4-name%22%20class%3D%22er%20entityLabel%22%3Efriends%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%2292.0625%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-4-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22114.578125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-5-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22114.578125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-5-name%22%20class%3D%22er%20entityLabel%22%3Euseful%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22114.578125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-5-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22137.09375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-6-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22137.09375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-6-name%22%20class%3D%22er%20entityLabel%22%3Efunny%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22137.09375%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C148.3515625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-6-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22159.609375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-7-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22159.609375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-7-name%22%20class%3D%22er%20entityLabel%22%3Ecool%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22159.609375%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C170.8671875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-7-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22182.125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-8-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22182.125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-8-name%22%20class%3D%22er%20entityLabel%22%3Efans%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22182.125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C193.3828125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-8-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22204.640625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-9-type%22%20class%3D%22er%20entityLabel%22%3Eint%5B%5D%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22204.640625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-9-name%22%20class%3D%22er%20entityLabel%22%3Eelite%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22204.640625%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C215.8984375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-9-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22227.15625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-10-type%22%20class%3D%22er%20entityLabel%22%3Efloat%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22227.15625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-10-name%22%20class%3D%22er%20entityLabel%22%3Eaverage_stars%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22227.15625%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C238.4140625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-10-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22249.671875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-11-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22249.671875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-11-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_hot%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22249.671875%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C260.9296875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-11-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22272.1875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-12-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22272.1875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-12-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_more%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22272.1875%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C283.4453125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-12-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22294.703125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-13-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22294.703125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-13-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_profile%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22294.703125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C305.9609375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-13-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22317.21875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-14-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22317.21875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-14-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_cute%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22317.21875%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C328.4765625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-14-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22339.734375%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-15-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22339.734375%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-15-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_list%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22339.734375%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C350.9921875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-15-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22362.25%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C373.5078125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-16-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22362.25%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C373.5078125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-16-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_note%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22362.25%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C373.5078125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-16-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22384.765625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C396.0234375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-17-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22384.765625%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C396.0234375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-17-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_plain%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22384.765625%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C396.0234375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-17-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22407.28125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C418.5390625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-18-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22407.28125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C418.5390625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-18-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_cool%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22407.28125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C418.5390625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-18-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22429.796875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C441.0546875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-19-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22429.796875%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C441.0546875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-19-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_funny%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22429.796875%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C441.0546875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-19-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22452.3125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C463.5703125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-20-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22452.3125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C463.5703125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-20-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_writer%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22452.3125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C463.5703125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-20-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2243.234375%22%20y%3D%22474.828125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C486.0859375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-21-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2299.96875%22%20y%3D%22474.828125%22%20x%3D%2243.234375%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2848.234375%2C486.0859375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-21-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_photos%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.5625%22%20y%3D%22474.828125%22%20x%3D%22143.203125%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28148.203125%2C486.0859375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-User-818c511a-4a32-5484-ba70-875d765a9175-attr-21-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20transform%3D%22translate%2820%2C98.8046875%20%29%22%20id%3D%22entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2%22%3E%3Crect%20height%3D%2269.546875%22%20width%3D%22118.625%22%20y%3D%220%22%20x%3D%220%22%20class%3D%22er%20entityBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20text-anchor%3A%20middle%3B%20font-size%3A%2012px%3B%22%20transform%3D%22translate%2859.3125%2C12.2578125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2%22%20class%3D%22er%20entityLabel%22%3ECheckin%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2224.515625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-1-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2224.515625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-1-name%22%20class%3D%22er%20entityLabel%22%3Ebusiness_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2224.515625%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-1-key%22%20class%3D%22er%20entityLabel%22%3EFK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2247.03125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-2-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2261.640625%22%20y%3D%2247.03125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-2-name%22%20class%3D%22er%20entityLabel%22%3Edate%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2247.03125%22%20x%3D%2297.390625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28102.390625%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Checkin-24698e1f-abf9-5bfe-bfd7-cb37857c4bd2-attr-2-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20transform%3D%22translate%28270.89453125%2C65.03125%20%29%22%20id%3D%22entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4%22%3E%3Crect%20height%3D%22137.09375%22%20width%3D%22152.296875%22%20y%3D%220%22%20x%3D%220%22%20class%3D%22er%20entityBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20text-anchor%3A%20middle%3B%20font-size%3A%2012px%3B%22%20transform%3D%22translate%2876.1484375%2C12.2578125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4%22%20class%3D%22er%20entityLabel%22%3ETip%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2224.515625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-1-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2295.3125%22%20y%3D%2224.515625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-1-name%22%20class%3D%22er%20entityLabel%22%3Etext%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2224.515625%22%20x%3D%22131.0625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28136.0625%2C35.7734375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-1-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2247.03125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-2-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2295.3125%22%20y%3D%2247.03125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-2-name%22%20class%3D%22er%20entityLabel%22%3Edate%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2247.03125%22%20x%3D%22131.0625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28136.0625%2C58.2890625%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-2-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2269.546875%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-3-type%22%20class%3D%22er%20entityLabel%22%3Eint%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2295.3125%22%20y%3D%2269.546875%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-3-name%22%20class%3D%22er%20entityLabel%22%3Ecompliment_count%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2269.546875%22%20x%3D%22131.0625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28136.0625%2C80.8046875%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-3-key%22%20class%3D%22er%20entityLabel%22%3E%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%2292.0625%22%20x%3D%220%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-4-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2295.3125%22%20y%3D%2292.0625%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-4-name%22%20class%3D%22er%20entityLabel%22%3Ebusiness_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%2292.0625%22%20x%3D%22131.0625%22%20class%3D%22er%20attributeBoxEven%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28136.0625%2C103.3203125%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-4-key%22%20class%3D%22er%20entityLabel%22%3EFK%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2235.75%22%20y%3D%22114.578125%22%20x%3D%220%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%285%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-5-type%22%20class%3D%22er%20entityLabel%22%3Estring%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2295.3125%22%20y%3D%22114.578125%22%20x%3D%2235.75%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%2840.75%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-5-name%22%20class%3D%22er%20entityLabel%22%3Euser_id%3C%2Ftext%3E%3Crect%20height%3D%2222.515625%22%20width%3D%2221.234375%22%20y%3D%22114.578125%22%20x%3D%22131.0625%22%20class%3D%22er%20attributeBoxOdd%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22dominant-baseline%3A%20middle%3B%20font-size%3A%2010.2px%3B%22%20transform%3D%22translate%28136.0625%2C125.8359375%29%22%20y%3D%220%22%20x%3D%220%22%20id%3D%22text-entity-Tip-281c7dc1-63bd-5831-b87d-6a8af7576bb4-attr-5-key%22%20class%3D%22er%20entityLabel%22%3EFK%3C%2Ftext%3E%3C%2Fg%3E%3Crect%20height%3D%2214.515625%22%20width%3D%2210.890625%22%20y%3D%22322.7985534667969%22%20x%3D%22393.7367858886719%22%20class%3D%22er%20relationshipLabelBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22text-anchor%3A%20middle%3B%20dominant-baseline%3A%20middle%3B%20font-size%3A%2012px%3B%22%20y%3D%22330.0563659667969%22%20x%3D%22399.1820983886719%22%20id%3D%22rel6%22%20class%3D%22er%20relationshipLabel%22%3Eof%3C%2Ftext%3E%3Crect%20height%3D%2214.515625%22%20width%3D%2212.609375%22%20y%3D%22278.475830078125%22%20x%3D%22665.3582763671875%22%20class%3D%22er%20relationshipLabelBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22text-anchor%3A%20middle%3B%20dominant-baseline%3A%20middle%3B%20font-size%3A%2012px%3B%22%20y%3D%22285.733642578125%22%20x%3D%22671.6629638671875%22%20id%3D%22rel7%22%20class%3D%22er%20relationshipLabel%22%3Eby%3C%2Ftext%3E%3Crect%20height%3D%2214.515625%22%20width%3D%2210.890625%22%20y%3D%22295.7034912109375%22%20x%3D%22249.5012969970703%22%20class%3D%22er%20relationshipLabelBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22text-anchor%3A%20middle%3B%20dominant-baseline%3A%20middle%3B%20font-size%3A%2012px%3B%22%20y%3D%22302.9613037109375%22%20x%3D%22254.9466094970703%22%20id%3D%22rel8%22%20class%3D%22er%20relationshipLabel%22%3Eof%3C%2Ftext%3E%3Crect%20height%3D%2214.515625%22%20width%3D%2212.625%22%20y%3D%22278.42083740234375%22%20x%3D%22490.2077331542969%22%20class%3D%22er%20relationshipLabelBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22text-anchor%3A%20middle%3B%20dominant-baseline%3A%20middle%3B%20font-size%3A%2012px%3B%22%20y%3D%22285.67864990234375%22%20x%3D%22496.5202331542969%22%20id%3D%22rel9%22%20class%3D%22er%20relationshipLabel%22%3Eby%3C%2Ftext%3E%3Crect%20height%3D%2214.515625%22%20width%3D%2210.890625%22%20y%3D%22312.7161865234375%22%20x%3D%2292.76653289794922%22%20class%3D%22er%20relationshipLabelBox%22%3E%3C%2Frect%3E%3Ctext%20style%3D%22text-anchor%3A%20middle%3B%20dominant-baseline%3A%20middle%3B%20font-size%3A%2012px%3B%22%20y%3D%22319.9739990234375%22%20x%3D%2298.21184539794922%22%20id%3D%22rel10%22%20class%3D%22er%20relationshipLabel%22%3Eof%3C%2Ftext%3E%3Ctext%20class%3D%22entityTitleText%22%20y%3D%22-25%22%20x%3D%22367.525390625%22%3EYelp%20Dataset%3C%2Ftext%3E%3C%2Fsvg%3E)

## Objective
```
80K     Dataset_User_Agreement.pdf
114M    yelp_academic_dataset_business.json
274M    yelp_academic_dataset_checkin.json
5.0G    yelp_academic_dataset_review.json
173M    yelp_academic_dataset_tip.json
3.2G    yelp_academic_dataset_user.json
```

Since the data files are too large to be processed manually, we will extract a **chronogical** subset of the Yelp Dataset with Spark for further analysis.
Additionally, some data transformations (eg. **denormalisation**) will be performed to simplify later analysis.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.getActiveSession()

## Load Dataset
Load the Yelp dataset with Spark:

In [ ]:
prefix = "yelp_academic_dataset_"
s3_bucket = "ntu-sc1015-yelp"
tables = [
    "business",
    "checkin",
    "review",
    "tip",
    "user",
]

dataset = {}
for table in tables:
    dataset[table] = spark.read.json(f"s3://{s3_bucket}/{prefix}{table}.json")

Prefix column names with to prevent name collisions when joining:

In [ ]:
for table in ["business", "user"]:
    for col in dataset[table].columns:
        if col != f"{table}_id":
            dataset[table] = dataset[table].withColumnRenamed(col, f"{table}_{col}")

Presort `Business` & `User` tables to speed up later sort merge joins

In [ ]:
business_df = dataset["business"].sort("business_id").cache()
user_df = dataset["user"].sort("user_id").cache()

## Reviews
Sample last `n_sample` reviews & denormalise review data.

In [ ]:
n_sample = 100000

(
    dataset["review"].withColumn("date", F.to_date("date"))
    # sample the n_sample last reviews in chronological order to ensure that reviews are contiguous over time.
    .sort("date", ascending=False)
    .limit(n_sample)
    # denormalise by joining use and business data to reviews
    # join business first as it is the smallest table
    .join(business_df, "business_id")
    .join(user_df, "user_id")
    # write as one parquet file back to s3
    .coalesce(1)
    .write
    .mode("overwrite")
    .parquet(f"s3a://{s3_bucket}/yelp_reviews.parquet")
)

## Tips
Sample last `n_sample` tips & denormalise Tips data.


In [ ]:
(
    dataset["tip"].withColumn("date", F.to_date("date"))
    # sample the n_sample last tips in chronological order to ensure that tips are contiguous over time.
    .sort("date", ascending=False)
    .limit(n_sample)
    # denormalise by joining use and business data to tips
    # join business first as it is the smallest table
    .join(business_df, "business_id")
    .join(user_df, "user_id")
    # write as one parquet file back to s3
    .coalesce(1)
    .write
    .mode("overwrite")
    .parquet(f"s3a://{s3_bucket}/yelp_tips.parquet")
)

## Check Ins
Check ins are represented as a list of comma-seperated timestamps in the dataset.
- Expand comma seperated list into individual check in rows.

In [ ]:
(
    dataset["checkin"]
    # expand comma delimited dates into individual rows
    .select("business_id", F.explode(F.split("date", ",").alias("date")))
    # write as one parquet file back to s3
    .coalesce(1)
    .write
    .mode("overwrite")
    .parquet(f"s3a://{s3_bucket}/yelp_checkins.parquet")
)